## **Face Recognition using Principal Component Analysis**
Recent advance in machine learning has made face recognition not a difficult problem. But in the previous, researchers have made various attempts and developed various skills to make computer capable of identifying people. One of the early attempt with moderate success is eigenface, which is based on linear algebra techniques.

In this tutorial, we will see how we can build a primitive face recognition system with some simple linear algebra technique such as principal component analysis.

After completing this tutorial, you will know:<br>
*1.* The development of eigenface technique<br>
*2.* How to use principal component analysis to extract<br>
*3.* characteristic images from an image dataset<br>
*4.* How to express any image as a weighted sum of the characteristic images<br>
*5.* How to compare the similarity of images from the weight of principal components<br>

*Tutorial overview*
This tutorial is divided into 3 parts; they are:<br>
*1.* Image and Face Recognition<br>
*2.* Overview of Eigenface<br>
*3.* Implementing Eigenface<br>

*Image and Face Recognition*<br>
In computer, pictures are represented as a matrix of pixels, with each pixel a particular color coded in some numerical values. It is natural to ask if computer can read the picture and understand what it is, and if so, whether we can describe the logic using matrix mathematics. To be less ambitious, people try to limit the scope of this problem to identifying human faces. An early attempt for face recognition is to consider the matrix as a high dimensional detail and we infer a lower dimension information vector from it, then try to recognize the person in lower dimension. It was necessary in the old time because the computer was not powerful and the amount of memory is very limited. However, by exploring how to compress image to a much smaller size, we developed a skill to compare if two images are portraying the same human face even if the pictures are not identical.<br>

In 1987, a paper by Sirovich and Kirby considered the idea that all pictures of human face to be a weighted sum of a few “key pictures”. Sirovich and Kirby called these key pictures the “eigenpictures”, as they are the eigenvectors of the covariance matrix of the mean-subtracted pictures of human faces. In the paper they indeed provided the algorithm of principal component analysis of the face picture dataset in its matrix form. And the weights used in the weighted sum indeed correspond to the projection of the face picture into each eigenpicture.<br>

In 1991, a paper by Turk and Pentland coined the term “eigenface”. They built on top of the idea of Sirovich and Kirby and use the weights and eigenpictures as characteristic features to recognize faces. The paper by Turk and Pentland laid out a memory-efficient way to compute the eigenpictures. It also proposed an algorithm on how the face recognition system can operate, including how to update the system to include new faces and how to combine it with a video capture system. The same paper also pointed out that the concept of eigenface can help reconstruction of partially obstructed picture.<br>

## **Overview of Eigenface**<br>
Before we jump into the code, let’s outline the steps in using eigenface for face recognition, and point out how some simple linear algebra technique can help the task.<br>

Assume we have a bunch of pictures of human faces, all in the same pixel dimension (e.g., all are r×c grayscale images). If we get M different pictures and **vectorize** each picture into L=r×c pixels, we can present the entire dataset as a L×M matrix (let’s call it matrix $A$), where each element in the matrix is the pixel’s grayscale value.<br>

Recall that principal component analysis (PCA) can be applied to any matrix, and the result is a number of vectors called the **principal components**. Each principal component has the length same as the column length of the matrix. The different principal components from the same matrix are orthogonal to each other, meaning that the vector dot-product of any two of them is zero. Therefore the various principal components constructed a vector space for which each column in the matrix can be represented as a linear combination (i.e., weighted sum) of the principal components.<br>

The way it is done is to first take $$C=A-a$$ where $a$ is the mean vector of the matrix $A$. So $C$ is the matrix that subtract each column of $A$ with the mean vector $a$. Then the covariance matrix is 
$$S=CC^T$$
from which we find its eigenvectors and eigenvalues. The principal components are these eigenvectors in decreasing order of the eigenvalues. Because matrix $S$ is a L×L matrix, we may consider to find the eigenvectors of a M×M matrix $C^TC$ instead as the eigenvector $\nu$ for $C^TC$ can be transformed into eigenvector $u$ of $C^TC$ by $u=C\nu$, except we usually prefer to write $u$ as normalized vector (i.e., norm of $u$ is 1).<br>

The physical meaning of the principal component vectors of $A$, or equivalently the eigenvectors of $S=CC^T$, is that they are the key directions that we can construct the columns of matrix $A$. The relative importance of the different principal component vectors can be inferred from the corresponding eigenvalues. The greater the eigenvalue, the more useful (i.e., holds more information about $A$) the principal component vector. Hence we can keep only the first K principal component vectors. If matrix 
$A$ is the dataset for face pictures, the first K principal component vectors are the top K most important “face pictures”. We call them the **eigenface** picture.<br>

For any given face picture, we can project its mean-subtracted version onto the eigenface picture using vector dot-product. The result is how close this face picture is related to the eigenface. If the face picture is totally unrelated to the eigenface, we would expect its result is zero. For the K eigenfaces, we can find K dot-product for any given face picture. We can present the result as **weights** of this face picture with respect to the eigenfaces. The weight is usually presented as a vector.

Conversely, if we have a weight vector, we can add up each eigenfaces subjected to the weight and reconstruct a new face. Let’s denote the eigenfaces as matrix $F$, which is a L×K matrix, and the weight vector $w$ is a column vector. Then for any $w$ we can construct the picture of a face as
$$z=Fw$$
where $z$ is resulted as a column vector of length L. Because we are only using the top K principal component vectors, we should expect the resulting face picture is distorted but retained some facial characteristic.

Since the eigenface matrix is constant for the dataset, a varying weight vector $w$ means a varying face picture. Therefore we can expect the pictures of the same person would provide similar weight vectors, even if the pictures are not identical. As a result, we may make use of the distance between two weight vectors (such as the L2-norm) as a metric of how two pictures resemble.

## **使用主成分分析进行人脸识别**

机器学习的最新进展使得人脸识别不再是一个困难的问题。但在过去，研究人员曾做出各种尝试并开发了各种技巧，使计算机能够识别人物。其中一种早期的尝试是特征脸（eigenface），它基于线性代数技术。

在本教程中，我们将看到如何使用一些简单的线性代数技术，如主成分分析，构建一个简单的人脸识别系统。

完成本教程后，您将了解以下内容：<br>
*1.* 特征脸技术的发展<br>
*2.* 如何使用主成分分析从图像数据集中提取特征图像<br>
*3.* 如何将任何图像表示为特征图像的加权和<br>
*4.* 如何根据主成分的权重比较图像的相似度<br>

**教程概览**<br>
本教程分为3部分：<br>
*1.* 图像和人脸识别<br>
*2.* 特征脸概述<br>
*3.* 实现特征脸<br>

**图像和人脸识别**<br>
在计算机中，图像被表示为像素矩阵，其中每个像素以一些数值形式编码为特定颜色。人们自然而然地会问计算机是否能够读取图像并理解其内容，如果可以，我们是否可以使用矩阵数学来描述这种逻辑。为了不那么雄心勃勃，人们尝试将这个问题的范围限制在识别人脸上。早期的人脸识别尝试是将矩阵视为一个高维细节，并从中推断出一个低维信息向量，然后尝试在低维中识别人物。在过去，这是必要的，因为计算机的性能有限，内存量非常有限。然而，通过探索如何将图像压缩到更小的尺寸，我们开发了一种技能，即使图像不完全相同，也可以比较两幅图像是否描绘了同一张人脸。<br>

1987年，Sirovich和Kirby的一篇论文考虑了人脸的所有图片都是少数“关键图片”的加权和的想法。Sirovich和Kirby将这些关键图片称为“特征图片”，因为它们是人脸图片的均值减去的图片的协方差矩阵的特征向量。在论文中，他们确实提供了面部图片数据集的主成分分析算法的矩阵形式。加权和中使用的权重确实对应于面部图片投影到每个特征图片中。<br>

1991年，Turk和Pentland的一篇论文首次提出了“特征脸”这个术语。他们在Sirovich和Kirby的思想基础上构建，并使用权重和特征图片作为识别人脸的特征。Turk和Pentland的论文提出了一种计算特征图片的内存高效的方法。它还提出了关于人脸识别系统如何运行的算法，包括如何更新系统以包含新的人脸以及如何将其与视频捕获系统结合使用。同一篇论文还指出了特征脸的概念可以帮助部分遮挡图片的重构。<br>

## **特征脸概述**<br>
在我们开始编码之前，让我们概述使用特征脸进行人脸识别的步骤，并指出一些简单的线性代数技术如何帮助完成任务。<br>

假设我们有一堆人脸图片，所有图片都具有相同的像素尺寸（例如，所有图片都是r×c灰度图像）。如果我们有M张不同的图片，并将每张图片**向量化**为L=r×c像素，我们可以将整个数据集表示为一个L×M矩阵（称为矩阵 $A$），其中矩阵中的每个元素都是像素的灰度值。<br>

回想一下，主成分分析（PCA）可以应用于任何矩阵，其结果是一些称为**主成分**的向量。每个主成分的长度与矩阵的列长度相同。来自同一矩阵的不同主成分彼此正交，这意味着它们中的任何两个向量的点积为零。因此，各个主成分构成了一个向量空间，矩阵中的每一列都可以表示为主成分的线性组合（即加权和）。<br>

这样做的方法是首先取$$C=A-a$$其中 $a$ 是矩阵 $A$ 的平均向量。因此，$C$ 是一个矩阵，它将矩阵 $A$ 的每一列都减去平均向量 $a$。然后，协方差矩阵为
$$S=CC^T$$
从中我们找到它的特征向量和特征值。主成分是这些特征向量，按特征值递减的顺序排列。因为矩阵 $S$ 是一个 L×L 矩阵，所以我们可以考虑找到一个 M×M 矩阵 $C^TC$ 的特征向量，而不是矩阵 $S=CC^T$ 的特征向量 $\nu$ 可以通过 $u=C\nu$ 转换为 $C^TC$ 的特征向量 $u$，除非我们通常更喜欢将 $u$ 写成归一化向量（即 $u$ 的范数为1）。<br>

矩阵 $A$ 的主成分向量（或等价地说是矩阵 $S=CC^T$ 的特征向量）的物理意义是它们是我们可以构造矩阵 $A$ 的列的关键方向。可以从相应的特征值推断出不同主成分向量的相对重要性。特征值越大，主成分向量越有用（即包含了更多关于 $A$ 的信息）。因此，我们只保留前 K 个主成分向量。如果矩阵 $A$ 是人脸图片的数据集，则前 K 个主成分向量是最重要的 K 个“特征脸图片”。我们将它们称为 **特征脸** 图片。<br>

对于任何给定的人脸图片，我们可以将其减去平均值后投影到特征脸图片上，使用向量点积。结果是这张人脸图片与特征脸的相关性有多大。如果人脸图片与特征脸完全不相关，我们期望其结果为零。对于 K 个特征脸，我们可以为任何给定的人脸图片找到 K 个点积。我们可以将结果表示为相对于特征脸的此人脸图片的**权重**。权重通常表示为一个向量。<br>

相反，如果我们有一个权重向量，我们可以根据权重添加每个特征脸并重构一个新的脸。让我们将特征脸表示为矩阵 $F$，它是一个 L×K 矩阵，权重向量 $w$ 是一个列向量。对于任何 $w$，我们都可以构造一张脸的图片
$$z=Fw$$
其中 $z$ 是长度为 L 的结果列向量。因为我们只使用了前 K 个主成分向量，所以我们应该期望生成的脸部图片会被扭曲但保留一些面部特征。<br>

由于特征脸矩阵对于数据集是恒定的，不同的权重向量 $w$ 意味着不同的脸部图片。因此，我们可以期望相同人的图片提供相似的权重向量，即使图片不完全相同。因此，我们可以利用两个权重向量之间的距离（例如 L2-范数）作为两个图片相似程度的度量。


In [1]:
import os
from PIL import Image

def convert_pgm_to_png(folder_path):
    # 遍历文件夹及其子文件夹
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.pgm'):
                pgm_path = os.path.join(root, file)
                # 构造输出的png文件路径
                png_path = os.path.splitext(pgm_path)[0] + '.png'
                # 打开pgm图像并转换成png格式
                with Image.open(pgm_path) as img:
                    img.save(png_path)
                print(f"Converted {pgm_path} to {png_path}")

# 调用函数并指定要转换的文件夹路径
folder_path="D:/OneDrive - email.szu.edu.cn/Manifold Learning/Principal component analysis/attface"
convert_pgm_to_png(folder_path)


Converted D:/OneDrive - email.szu.edu.cn/Manifold Learning/Principal component analysis/attface\s1\1.pgm to D:/OneDrive - email.szu.edu.cn/Manifold Learning/Principal component analysis/attface\s1\1.png
Converted D:/OneDrive - email.szu.edu.cn/Manifold Learning/Principal component analysis/attface\s1\10.pgm to D:/OneDrive - email.szu.edu.cn/Manifold Learning/Principal component analysis/attface\s1\10.png
Converted D:/OneDrive - email.szu.edu.cn/Manifold Learning/Principal component analysis/attface\s1\2.pgm to D:/OneDrive - email.szu.edu.cn/Manifold Learning/Principal component analysis/attface\s1\2.png
Converted D:/OneDrive - email.szu.edu.cn/Manifold Learning/Principal component analysis/attface\s1\3.pgm to D:/OneDrive - email.szu.edu.cn/Manifold Learning/Principal component analysis/attface\s1\3.png
Converted D:/OneDrive - email.szu.edu.cn/Manifold Learning/Principal component analysis/attface\s1\4.pgm to D:/OneDrive - email.szu.edu.cn/Manifold Learning/Principal component analysis/